# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`
---

In [84]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [85]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [86]:
# confirm that our new database was created
# List all databases in the MongoDB server
databases = mongo.list_database_names()

# Print the list of databases
pprint(databases)


['admin',
 'autosaurus',
 'classDB',
 'config',
 'fruits_db',
 'local',
 'met',
 'uk_food']


In [87]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [88]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in 'uk_food' database:")
pprint(collections)

Collections in 'uk_food' database:
['establishments']


In [89]:
# review a document in the establishments collection
# Assign the establishments collection to a variable
establishments = db['establishments']

# Review one document in the establishments collection
print("\nOne document from the 'establishments' collection:")
pprint(establishments.find_one())


One document from the 'establishments' collection:
{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 551803,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677705108805a06b93492bcc'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/551803',
            'rel': 'self'}],
 '

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [90]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [91]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavours)

InsertOneResult(ObjectId('677705216983792fb758e699'), acknowledged=True)

In [92]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677705216983792fb758e699'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [93]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Query to find the BusinessTypeID and BusinessType for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Select only the BusinessTypeID and BusinessType fields
result = establishments.find_one(query, {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1})

# Display the result
pprint(result)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [94]:
# Update the new restaurant with the correct BusinessTypeID
# Check if result contains the BusinessTypeID
if result and "BusinessTypeID" in result:
    # Extract the BusinessTypeID
    business_type_id = result["BusinessTypeID"]

    # Update the "Penang Flavours" restaurant with the found BusinessTypeID
    establishments.update_one(
        {"BusinessName": "Penang Flavours"},  # Filter by restaurant name
        {"$set": {"BusinessTypeID": business_type_id}}  # Set the new BusinessTypeID
    )
    print("Restaurant 'Penang Flavours' updated with BusinessTypeID successfully!")
else:
    print("BusinessTypeID not found for 'Restaurant/Cafe/Canteen'.")


Restaurant 'Penang Flavours' updated with BusinessTypeID successfully!


In [95]:
# Confirm that the new restaurant was updated
updated_document = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_document)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677705216983792fb758e699'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [96]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the number of documents that match the query
count = establishments.count_documents(query)

# Print the count
print(f"Number of documents with LocalAuthorityName 'Dover': {count}")


Number of documents with LocalAuthorityName 'Dover': 994


In [97]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many(query)

In [98]:
# Check if any remaining documents include Dover
remaining_documents = establishments.find(query)

# Check if there are any remaining documents and print the result
remaining_count = sum(1 for _ in remaining_documents)

if remaining_count > 0:
    print(f"There are {remaining_count} documents with LocalAuthorityName 'Dover' remaining.")
else:
    print("No documents with LocalAuthorityName 'Dover' remain.")

No documents with LocalAuthorityName 'Dover' remain.


In [99]:
# Check that other documents remain with 'find_one'
# Query to find a document where LocalAuthorityName is not "Dover"
query = {"LocalAuthority.Name": {"$ne": "Dover"}}

# Find one document that matches the query
remaining_document = establishments.find_one(query)
pprint(remaining_document)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677705108805a06b93492eb2'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [101]:
# Change the data type from String to Decimal for longitude and latitude
from bson import Decimal128
from decimal import Decimal

# Query to find documents where latitude and longitude are stored as strings
query = {
    "geocode.longitude": {"$type": "string"},
    "geocode.latitude": {"$type": "string"}
}

# Initialize counter for updated documents
updated_count = 0

# Loop through all documents where geocode longitude and latitude are stored as strings
for doc in establishments.find(query):
    longitude_str = doc["geocode"].get("longitude")
    latitude_str = doc["geocode"].get("latitude")

    # Convert longitude and latitude from string to Decimal if they are strings
    if isinstance(longitude_str, str):
        longitude_decimal = Decimal(longitude_str)
    if isinstance(latitude_str, str):
        latitude_decimal = Decimal(latitude_str)

    # Update the document with Decimal values in the geocode dictionary
    establishments.update_many(
        {"_id": doc["_id"]},
        {"$set": {
            "geocode.longitude": Decimal128(longitude_decimal),
            "geocode.latitude": Decimal128(latitude_decimal)
        }}
    )
    
    # Increment the counter after each update
    updated_count += 1

# Print the number of documents updated
print(f"Number of documents updated: {updated_count}")



Number of documents updated: 3387


In [107]:
#check that the Geocodes are now decimal numbers
geocode_result = establishments.find_one({}, {"geocode": 1})  # Get only the geocode field
pprint(geocode_result)

{'_id': ObjectId('677705108805a06b93492eb2'),
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')}}


Use `update_many` to convert `RatingValue` to integer numbers.

In [102]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [103]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$in": ["1", "2", "3", "4", "5"]}},  # Select rating values that are valid strings
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  # Convert string values to integers
)

UpdateResult({'n': 34629, 'nModified': 34629, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [108]:
# Check that the rating value are now numbers
rating_result = establishments.find_one({}, {"RatingValue": 1})  # Get only the RatingValue field
pprint(rating_result)

{'RatingValue': 4, '_id': ObjectId('677705108805a06b93492eb2')}
